In [ ]:
import os
import sys
import pathlib
import json

import numpy as np
import pandas as pd

import logging
logging.basicConfig(stream=sys.stdout, format='%(asctime)-15s %(message)s',
                level=logging.DEBUG, datefmt=None)
logger = logging.getLogger("db-ingest")

In [ ]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [ ]:
users_df = pd.read_csv('../data/users.csv', index_col=0)
users_df.head(3)
events_df = pd.read_csv('../data/events.csv', index_col=0)
events_df.head(3)

In [ ]:
users_df = users_df.loc[:, ['id', 'created', 'storage', 'created']]
users_df.head()
events_df = events_df.loc[:, ['id', 'direction', 'size', 'time.backend', 'timestamp']]
events_df.head()

In [ ]:
### TRANSFER SPEED
KB = 1e3
mS = 1e-3
events_df['size_KB'] = events_df['size'].apply(lambda x: x / KB)
events_df['time.backend [s]'] = events_df['time.backend'].apply(lambda x: x * mS)
events_df['transfer_speed [KB/s]'] = events_df['size_KB'] / events_df['time.backend.seconds']
events_df.loc[:5, ['size', 'size_KB', 'time.backend', 'time.backend [s]', 'transfer_speed [KB/s]']]

In [ ]:
events_df.to_csv('../data/events_post.csv')
users_df.to_csv('../data/users_post.csv')